<a href="https://colab.research.google.com/github/Rikupro-creator/LLM-prompt-recovery/blob/main/Copy_of_baseline_LLm_prompt_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'llm-prompt-recovery:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67121%2F7806901%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T150906Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D787180bc2983c51c86639eba8d8c5c7bdb7e77d2c836f051174bd8d219bedb0446533b3cf2b61bd9a3190844022cb938a47584f3ee25cd624de3f8b342841ab9afe2dc71d178cdbf4ccd1c2ffcfdcd6a52ac1a4bd3aec4a64880a0deabf0170fc1528f1a228678cb070f817a9f2859f831c9afd478b799f65ae6e722a382d4115765e3cfdc57f61f8b96c3e154cee1cea68c007203b23e21edbd6e5cfe17b3a4d8fb5183358a9cfe1cb552c8151fd51b8cd7dce749541b1649524d8c14d15096631568008d392ff936a9535c78c4140290694206fc355d35ac6a0452674f22acba34a748ac2edcec5356064e8138f4f68884e0f0f8c5e797f2853bb196c216de,3000-rewritten-texts-prompt-recovery-challenge:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4518936%2F7733314%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T150906Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4fd19fef2bcf7ff6ddc8d07856bfbedb116563ecb51a73361d647e961ffd1b2a8b97535c942fc5a0d65df9311f3a3e48631af802eb2abc9908e0d174fa7929b6a56a1a44630285f68b6dbff1094ad73cfcbcb3c2bc55f7284f12739184e0a5c712a276d52c856bf0440706ea4e758e56d19222f08e1a2da28cbf2fe7fe3a87a570c569ecc8e7dbee8b6ccb79fb5cc7c100f61244f231ca9affe96de3ef9f3f029f8dac256ce1e26ba4eb977f5fd7b5aa74f1d7a60e7cbf58439724a5e743b8d9ac9b928754953062b424bc7abc1f32dfe1cd74f0c11b409b65c22d380b4e318224fc46dd2993ef818a0b3034d5cd3708ffa6f2c8c962fceedea9477637605c52'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# LLM prompt model

In [ ]:
# importing the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [ ]:
train=pd.read_csv('/kaggle/input/llm-prompt-recovery/train.csv')
test=pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

In [ ]:
train.head()

In [ ]:
print(train['original_text'])

In [ ]:
data=pd.read_csv('/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv')
data.columns

In [ ]:
def extract_keywords(sentence):
    # Load the English tokenizer, tagger, parser, NER, and word vectors
    nlp = spacy.load("en_core_web_sm")
    # Process the sentence using spaCy
    doc = nlp(sentence)

    # Initialize lists to store keywords and keyword phrases
    keywords = []
    keyword_phrases = []

    for chunk in doc.noun_chunks:
        if len(chunk) == 1:
            token = chunk[0]
            if token.pos_ in ["NOUN", "VERB", "ADJ"] or token.ent_type_:
                keywords.append(token.text)

    return keywords

In [ ]:
import spacy
import string
import re
from nltk import tokenize

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

def remove_punctuation(text):
    res = re.sub(r'[^\w\s]', '', str(text))
    return res
def remove_prep_art(text):
    # Remove prepositions and articles
    doc = nlp(text)
    tokens = [token.text for token in doc if token.pos_ not in ['DET', 'ADP']]
    return ' '.join(tokens)
def get_first_sentence(paragraph):
    # Split the paragraph into sentences
    doc = nlp(paragraph)

    # Extract the first sentence
    for sentence in doc.sents:
        sentence = sentence.text.strip()
        if sentence:
            # Check if the sentence is enquoted
            if sentence.startswith("**") and sentence.endswith("**"):
                delimiters = ",:;'"
                for char in delimiters:
                    if char in sentence:
                        return remove_punctuation(sentence)
            elif sentence.startswith("##"):
                sentence_parts = re.split(r'[{}]+'.format(string.punctuation.replace("'", '')), sentence)
                sentence_parts = [part.strip() for part in sentence_parts if part.strip()]
                return sentence_parts[0]
            elif sentence.startswith("Sure, here's"):
                sentence = sentence[len("Sure, here's"):].strip()
                words = ['transformed', 'converted', 'rewritten', 'transform', 'convert', 'rewrite']
                for word in words:
                    if word in sentence:
                        index = sentence.find(word)
                        content = sentence[index + len(word):].strip()
                        # Remove prepositions and articles after the keyword
                        return remove_prep_art(content)
            else:
                # Extract statements before the first colon or comma
                parts = re.split(r'[:;]', sentence)
                statement = parts[0].strip() if len(parts) > 1 else sentence
                parts = re.split(r'[.:]', statement)
                return remove_punctuation(extract_keywords(parts[0].strip()))
    return None

# Example usage
first_sentence = get_first_sentence(data['rewritten_text'][4])
print("First sentence:", first_sentence)


In [ ]:
remove_punctuation('**JHklsggf')

In [ ]:
example=pd.read_csv('/kaggle/input/llm-prompt-recovery/sample_submission.csv')

In [ ]:
p=data['rewritten_text'][18]



In [ ]:
# Retrieve keywords for each paragraph in test['rewritten_text']
keywords = []
for paragraph in tqdm(test['rewritten_text']):
    first_sentence_keywords = get_first_sentence(paragraph)
    print(first_sentence)
    if first_sentence_keywords:
        keywords.append(first_sentence_keywords)
    else:
        keywords.append("")  # If no keywords found, append an empty string

# Generate rewritten sentences
rewrite = ['rewrite this essay using the style ' + i if i else "rewrite this text" for i in keywords]


In [ ]:
rewrite

In [ ]:
test['rewritten_text']

In [ ]:
submission=pd.DataFrame({
    'id':test.id,
    'rewrite_prompt':rewrite
})

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()